In [ ]:
!pip install tensorflow==2.15.0 pandas numpy scikit-learn tensorboard matplotlib scikeras nltk

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3

In [ ]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd


data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as f:
  f.write(data)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
with open('hamlet.txt','r') as file:
  text = file.read().lower()

tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(total_words)


In [ ]:
input_sequences=[]
for line in text.split('\n'):
  #print("line",line)
  tokenlist=tokenizer.texts_to_sequences([line])[0]
  #print("tokenlist",tokenlist)
  for i in range(1,len(tokenlist)):
    n_gram_sequence=tokenlist[:i+1]
    #print("n_gram_sequence",n_gram_sequence)
    input_sequences.append(n_gram_sequence)

  #print(input_sequences)


In [ ]:
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
print(input_sequences)

In [ ]:
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]


In [ ]:
x

In [ ]:
y

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
# model.add(Dense(total_words/2,activation='relu'))
model.add(Dense(total_words,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(X_train,y_train,epochs=300,validation_data=(X_test,y_test),verbose=1)

In [ ]:
def generate_text(seed_text,model,max_sequence_len):
  token_list=tokenizer.texts_to_sequences([seed_text])[0]
  token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
  predicted=model.predict(token_list,verbose=0)
  predicted_word_index=np.argmax(predicted,axis=1)
  for word,index in tokenizer.word_index.items():
    if index==predicted_word_index:
      return word
  return None

In [ ]:
sample_text='i like to'
max_sequence_len=model.input_shape[1]+1
next_word=generate_text(sample_text,model,max_sequence_len)
print(next_word)

In [ ]:
model.save('hamlet_model.h5')


In [ ]:
import pickle
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)